In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np

import warnings
import datetime as dt
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [2]:
events = pd.read_csv('data/events_up_to_01062018.csv', low_memory = False)
frecuencia = events[['person']]
frecuencia['cant_eventos'] = 1
frecuencia = pd.DataFrame({'cant_eventos': frecuencia.groupby('person').aggregate(sum)['cant_eventos']}).reset_index()
events = events.merge(frecuencia, on = 'person', how = 'inner')
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
3,2018-05-18 00:11:56,checkout,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9


In [3]:
del frecuencia

In [4]:
# prediction_set.person.describe()

In [5]:
# training_set.person.describe()

In [6]:
sets = events
cols = ['event','fecha','person', 'sku', 'model', 'condition', 'storage', 'color','cant_eventos']
# cols_visit = ['event','fecha','person','new_vs_returning','city','region','country','device_type','screen_resolution','operating_system_version','browser_version','cant_eventos','url', 'skus', 'search_term', 'staticpage', 'campaign_source', 'search_engine', 'channel']
cols_visit = ['event','fecha','person','new_vs_returning','city','region','country','device_type','screen_resolution','operating_system_version','browser_version','cant_eventos']
cols_x = ['fecha_x','person','sku_x', 'model_x', 'condition_x', 'storage_x', 'color_x','cant_eventos_x','compras_prod_x','vistas_prod_x', 'otros_x']
rename = {'fecha_x': 'fecha', 'model_x': 'model','sku_x':'sku', 'condition_x': 'condition', 'storage_x': 'storage', 'color_x': 'color','cant_eventos_x': 'cant_eventos', 'compras_prod_x': 'compras_prod', 'vistas_prod_x': 'vistas_prod' ,'otros_x': 'otros'}
# cols_x = ['fecha_x','person','sku_x', 'model_x', 'condition_x', 'storage_x', 'color_x','cant_eventos_x','compras_prod_x','vistas_prod_x', 'otros_x']
# rename = {'event_x': 'event', 'fecha_x': 'fecha', 'model_x': 'model','sku_x':'sku', 'condition_x': 'condition', 'storage_x': 'storage', 'color_x': 'color','cant_eventos_x': 'cant_eventos'}

In [7]:
del events

# Set Datos

# #Analisis TP1

In [8]:
sets.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'cant_eventos'],
      dtype='object')

In [9]:
#Cambiamos el tipo de dato de fecha de timestamp a datetime para un mejor analisis y renombramos a "fecha"
sets[['timestamp']] = sets[['timestamp']].apply(pd.to_datetime)
sets.rename({
    'timestamp' : 'fecha',
}, axis=1, inplace=True)
# sets.head()

vamos a crear las columnas compras_prod, vistas_prod, otros para productos y visitas_sitio para persona, de la forma de One Hot Encoder

In [10]:
#Detalles de Productos que fueron comprados
productos = sets.loc[:, cols]
productos_comprados = productos[(productos['event'] == 'conversion') | (productos['event'] == 'checkout')]
productos_comprados.drop(['event'], inplace=True, axis=1)
productos_comprados['compras_prod'] = 1
productos_comprados['vistas_prod'] = 1
productos_comprados['otros'] = 1
frecuencia_compras = productos_comprados[['person']]
frecuencia_compras['cant_compras'] = 1
frecuencia_compras = pd.DataFrame({'cant_compras': frecuencia_compras.groupby('person').aggregate(sum)['cant_compras']}).reset_index()
productos_comprados.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod,otros
3,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9,1,1,1
45,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,1,1,1
121,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,567,1,1,1
154,2018-05-10 00:39:51,0297fc1e,7029.0,iPhone 6S Plus,Bom,16GB,Cinza espacial,567,1,1,1
378,2018-02-25 18:38:00,0297fc1e,1342.0,iPhone 6,Muito Bom,64GB,Dourado,567,1,1,1


In [11]:
productos_comprados.count()

fecha           72406
person          72406
sku             72406
model           72406
condition       72406
storage         72406
color           72406
cant_eventos    72406
compras_prod    72406
vistas_prod     72406
otros           72406
dtype: int64

In [12]:
frecuencia_compras.head()

,person,cant_compras
0,0008ed71,3
1,00091926,2
2,000ba417,7
3,000c79fe,1
4,000e4d9e,1


In [13]:
# productos_comprados = productos_comprados.drop_duplicates()
# productos_comprados.count()

In [14]:
#Detalles de Productos que fueron visitados
productos_visitados = productos.loc[productos.event == 'viewed product']
productos_visitados.drop(['event'], inplace=True, axis=1)
productos_visitados['compras_prod'] = 0
productos_visitados['vistas_prod'] = 1
productos_visitados['otros'] = 1
frecuencia_vistas = productos_visitados[['person']]
frecuencia_vistas['cant_vistas'] = 1
frecuencia_vistas = pd.DataFrame({'cant_vistas': frecuencia_vistas.groupby('person').aggregate(sum)['cant_vistas']}).reset_index()
productos_visitados.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod,otros
0,2018-05-18 00:11:59,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9,0,1,1
1,2018-05-18 00:30:30,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9,0,1,1
4,2018-05-18 00:11:35,4886f805,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,9,0,1,1
6,2018-05-18 00:11:53,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9,0,1,1
9,2018-05-18 00:11:27,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,1,1


In [15]:
productos_visitados.count()

fecha           1248124
person          1248124
sku             1248124
model           1248124
condition       1248124
storage         1248124
color           1248124
cant_eventos    1248124
compras_prod    1248124
vistas_prod     1248124
otros           1248124
dtype: int64

In [16]:
frecuencia_vistas.head()

,person,cant_vistas
0,00091926,372
1,00091a7a,3
2,000ba417,153
3,000c79fe,3
4,000e4d9e,339


In [17]:
# productos_visitados = productos_comprados.drop_duplicates()
# productos_visitados.count()

In [18]:
# Eliminamos de productos visitados las personas que estan en productos comprados
productos_visitados = productos_visitados.merge(productos_comprados, on='person', how='left')
productos_visitados = productos_visitados[productos_visitados['fecha_y'].isnull()]
productos_visitados = productos_visitados.loc[:,cols_x]
productos_visitados.rename(columns=rename, inplace=True)
productos_visitados.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod,otros
22980,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,35,0,1,1
22981,2018-05-18 00:25:27,686c49c9,9946.0,iPhone 7,Excelente,128GB,Preto Matte,35,0,1,1
22982,2018-05-18 01:05:31,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,35,0,1,1
22983,2018-05-15 20:11:48,686c49c9,10030.0,iPhone 7,Excelente,128GB,Prateado,35,0,1,1
22984,2018-05-15 20:10:28,686c49c9,6845.0,iPhone 6S,Excelente,16GB,Dourado,35,0,1,1


In [19]:
otros = sets.loc[:, cols]
otros = otros[(sets['event'] == 'brand listing') | (sets['event'] == 'generic listing') | (sets['event'] == 'lead') | (sets['event'] == 'staticpage') ]                                                                      
otros.drop(['event'], inplace=True, axis=1)
otros['compras_prod'] = 0
otros['vistas_prod'] = 0
otros['otros'] = 1
frecuencia_otros = otros[['person']]
frecuencia_otros['cant_otros'] = 1
frecuencia_otros = pd.DataFrame({'cant_otros': frecuencia_otros.groupby('person').aggregate(sum)['cant_otros']}).reset_index()
otros.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod,otros
7,2018-05-18 00:07:22,4886f805,NaN,NaN,NaN,NaN,NaN,9,0,0,1
47,2018-05-14 23:50:23,ad93850f,NaN,NaN,NaN,NaN,NaN,65,0,0,1
48,2018-05-14 23:50:44,ad93850f,NaN,NaN,NaN,NaN,NaN,65,0,0,1
49,2018-05-14 23:51:11,ad93850f,NaN,NaN,NaN,NaN,NaN,65,0,0,1
50,2018-05-14 23:51:42,ad93850f,NaN,NaN,NaN,NaN,NaN,65,0,0,1


In [20]:
otros.count()

fecha           388672
person          388672
sku                  0
model              983
condition            0
storage              0
color                0
cant_eventos    388672
compras_prod    388672
vistas_prod     388672
otros           388672
dtype: int64

In [21]:
frecuencia_otros.head()

,person,cant_otros
0,0008ed71,1
1,00091926,25
2,00091a7a,5
3,000ba417,38
4,000c79fe,1


In [22]:
# otros = otros.drop_duplicates()
# otros.count()

In [23]:
# Eliminamos de otros las personas que estan en productos comprados
otros = otros.merge(productos_comprados, on='person', how='left')
otros = otros[otros['fecha_y'].isnull()]
otros = otros.loc[:,cols_x]
otros.rename(columns=rename, inplace=True)
# otros.head()

In [24]:
# Eliminamos de otros las personas que estan en productos visitados
otros = otros.merge(productos_visitados, on='person', how='left')
otros = otros[otros['fecha_y'].isnull()]
otros = otros.loc[:,cols_x]
otros.rename(columns=rename, inplace=True)
otros.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod,otros
119795,2018-05-28 12:36:14,df80a783,NaN,NaN,NaN,NaN,NaN,11,0,0,1
119796,2018-05-28 22:28:24,df80a783,NaN,NaN,NaN,NaN,NaN,11,0,0,1
119797,2018-05-29 14:47:28,df80a783,NaN,NaN,NaN,NaN,NaN,11,0,0,1
122010,2018-05-29 20:56:57,7a3184e8,NaN,NaN,NaN,NaN,NaN,3,0,0,1
122011,2018-05-29 19:45:16,f33e2cc5,NaN,NaN,NaN,NaN,NaN,4,0,0,1


In [25]:
#Rellanamos las columnas con valores promedios
otros['sku'] = labelencoder.fit_transform(otros['person'])
otros['model'] = 'Samsung Galaxy S7'
otros['condition'] = 'Excelente'
otros['storage'] = '16GB'
otros['color'] = 'Preto'
otros.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod,otros
119795,2018-05-28 12:36:14,df80a783,412,Samsung Galaxy S7,Excelente,16GB,Preto,11,0,0,1
119796,2018-05-28 22:28:24,df80a783,412,Samsung Galaxy S7,Excelente,16GB,Preto,11,0,0,1
119797,2018-05-29 14:47:28,df80a783,412,Samsung Galaxy S7,Excelente,16GB,Preto,11,0,0,1
122010,2018-05-29 20:56:57,7a3184e8,217,Samsung Galaxy S7,Excelente,16GB,Preto,3,0,0,1
122011,2018-05-29 19:45:16,f33e2cc5,441,Samsung Galaxy S7,Excelente,16GB,Preto,4,0,0,1


In [26]:
#Caracterisitcas del dispositivos con el que el usuario visito el sitio
visitas = sets.loc[:,cols_visit]
features_usuario = visitas.loc[visitas.event == 'visited site']
features_usuario.drop(['event'], inplace=True, axis=1)
features_usuario['visitas_sitio'] = 1
frecuencia_visitas = features_usuario[['person']]
frecuencia_visitas['cant_visitas_sitio'] = 1
frecuencia_visitas = pd.DataFrame({'cant_visitas_sitio': frecuencia_visitas.groupby('person').aggregate(sum)['cant_visitas_sitio']}).reset_index()
features_usuario.head()

,fecha,person,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos,visitas_sitio
8,2018-05-18 00:07:22,4886f805,New,Cantagalo,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,9,1
69,2018-05-14 23:50:22,ad93850f,New,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,1
70,2018-05-16 02:48:13,ad93850f,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,1
71,2018-05-18 00:11:26,ad93850f,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,1
72,2018-05-18 22:11:46,ad93850f,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,1


In [27]:
features_usuario.count()

fecha                       204069
person                      204069
new_vs_returning            204069
city                        204069
region                      204069
country                     204069
device_type                 204069
screen_resolution           204066
operating_system_version    204069
browser_version             204069
cant_eventos                204069
visitas_sitio               204069
dtype: int64

In [28]:
del productos
del visitas

In [29]:
user_prod_vistas = productos_visitados.merge(features_usuario, on="person", how="left")
user_prod_vistas.head()

,fecha_x,person,sku,model,condition,storage,color,cant_eventos_x,compras_prod,vistas_prod,...,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos_y,visitas_sitio
0,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,35,0,1,...,New,São Paulo,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,35.0,1.0
1,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,35,0,1,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,35.0,1.0
2,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,35,0,1,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,35.0,1.0
3,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,35,0,1,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,35.0,1.0
4,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,35,0,1,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,35.0,1.0


In [30]:
user_prod_vistas.count()

fecha_x                     1210149
person                      1210149
sku                         1210149
model                       1210149
condition                   1210149
storage                     1210149
color                       1210149
cant_eventos_x              1210149
compras_prod                1210149
vistas_prod                 1210149
otros                       1210149
fecha_y                     1209999
new_vs_returning            1209999
city                        1209999
region                      1209999
country                     1209999
device_type                 1209999
screen_resolution           1209999
operating_system_version    1209999
browser_version             1209999
cant_eventos_y              1209999
visitas_sitio               1209999
dtype: int64

In [31]:
user_prod_comprados = productos_comprados.merge(features_usuario, on="person", how="left")
user_prod_comprados.head()

,fecha_x,person,sku,model,condition,storage,color,cant_eventos_x,compras_prod,vistas_prod,...,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos_y,visitas_sitio
0,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9,1,1,...,New,Cantagalo,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,9.0,1.0
1,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,1,1,...,New,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0
2,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,1,1,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0
3,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,1,1,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0
4,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,1,1,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0


In [32]:
user_prod_comprados.count()

fecha_x                     1051711
person                      1051711
sku                         1051711
model                       1051711
condition                   1051711
storage                     1051711
color                       1051711
cant_eventos_x              1051711
compras_prod                1051711
vistas_prod                 1051711
otros                       1051711
fecha_y                     1050846
new_vs_returning            1050846
city                        1050846
region                      1050846
country                     1050846
device_type                 1050846
screen_resolution           1050832
operating_system_version    1050846
browser_version             1050846
cant_eventos_y              1050846
visitas_sitio               1050846
dtype: int64

In [33]:
otros = otros.merge(features_usuario, on="person", how="outer")
otros.head()

,fecha_x,person,sku,model,condition,storage,color,cant_eventos_x,compras_prod,vistas_prod,...,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos_y,visitas_sitio
0,2018-05-28 12:36:14,df80a783,412.0,Samsung Galaxy S7,Excelente,16GB,Preto,11.0,0.0,0.0,...,New,Timon,Maranhao,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,11.0,1.0
1,2018-05-28 12:36:14,df80a783,412.0,Samsung Galaxy S7,Excelente,16GB,Preto,11.0,0.0,0.0,...,Returning,Contagem,Minas Gerais,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,11.0,1.0
2,2018-05-28 12:36:14,df80a783,412.0,Samsung Galaxy S7,Excelente,16GB,Preto,11.0,0.0,0.0,...,Returning,Teresina,Piaui,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,11.0,1.0
3,2018-05-28 12:36:14,df80a783,412.0,Samsung Galaxy S7,Excelente,16GB,Preto,11.0,0.0,0.0,...,Returning,Teresina,Piaui,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,11.0,1.0
4,2018-05-28 22:28:24,df80a783,412.0,Samsung Galaxy S7,Excelente,16GB,Preto,11.0,0.0,0.0,...,New,Timon,Maranhao,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,11.0,1.0


In [34]:
otros.count()

fecha_x                       7186
person                      210502
sku                           7186
model                         7186
condition                     7186
storage                       7186
color                         7186
cant_eventos_x                7186
compras_prod                  7186
vistas_prod                   7186
otros                         7186
fecha_y                     210481
new_vs_returning            210481
city                        210481
region                      210481
country                     210481
device_type                 210481
screen_resolution           210478
operating_system_version    210481
browser_version             210481
cant_eventos_y              210481
visitas_sitio               210481
dtype: int64

In [35]:
del features_usuario
del productos_comprados
del productos_visitados

In [36]:
#concateno
frames = [user_prod_comprados, user_prod_vistas, otros]
sets = pd.concat(frames)
sets.head()

,fecha_x,person,sku,model,condition,storage,color,cant_eventos_x,compras_prod,vistas_prod,...,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos_y,visitas_sitio
0,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9.0,1.0,1.0,...,New,Cantagalo,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,9.0,1.0
1,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65.0,1.0,1.0,...,New,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0
2,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65.0,1.0,1.0,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0
3,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65.0,1.0,1.0,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0
4,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65.0,1.0,1.0,...,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0


In [37]:
del frames
del user_prod_comprados
del user_prod_vistas
del otros

In [38]:
sets.count()

fecha_x                     2269046
person                      2472362
sku                         2269046
model                       2269046
condition                   2269046
storage                     2269046
color                       2269046
cant_eventos_x              2269046
compras_prod                2269046
vistas_prod                 2269046
otros                       2269046
fecha_y                     2471326
new_vs_returning            2471326
city                        2471326
region                      2471326
country                     2471326
device_type                 2471326
screen_resolution           2471309
operating_system_version    2471326
browser_version             2471326
cant_eventos_y              2471326
visitas_sitio               2471326
dtype: int64

In [39]:
# Agregamos las frecuencias de cada persona de compras, vistas, visitas sitio y otros
sets = sets.merge(frecuencia_compras, on = 'person', how = 'left')
sets = sets.merge(frecuencia_vistas, on = 'person', how = 'left')
sets = sets.merge(frecuencia_otros, on = 'person', how = 'left')
sets = sets.merge(frecuencia_visitas, on = 'person', how = 'left')
sets.head()

,fecha_x,person,sku,model,condition,storage,color,cant_eventos_x,compras_prod,vistas_prod,...,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos_y,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio
0,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9.0,1.0,1.0,...,Smartphone,360x640,Android 7,Chrome Mobile 64.0,9.0,1.0,1.0,4.0,1.0,1.0
1,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65.0,1.0,1.0,...,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0,1.0,20.0,22.0,5.0
2,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65.0,1.0,1.0,...,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0,1.0,20.0,22.0,5.0
3,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65.0,1.0,1.0,...,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0,1.0,20.0,22.0,5.0
4,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65.0,1.0,1.0,...,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65.0,1.0,1.0,20.0,22.0,5.0


In [40]:
del frecuencia_compras
del frecuencia_vistas
del frecuencia_otros
del frecuencia_visitas
sets['cant_compras'] = sets['cant_compras'].fillna(value=0) 
sets['cant_vistas'] = sets['cant_vistas'].fillna(value=0) 
sets['cant_otros'] = sets['cant_otros'].fillna(value=0) 
sets['cant_visitas_sitio'] = sets['cant_visitas_sitio'].fillna(value=0) 

In [41]:
# Rellenamos compras_prod  vistas_prod  otros y visitas_sitio con 0
sets['compras_prod'] = sets['compras_prod'].fillna(value=0) 
sets['vistas_prod'] = sets['vistas_prod'].fillna(value=0) 
sets['otros'] = sets['otros'].fillna(value=0) 
sets['visitas_sitio'] = sets['visitas_sitio'].fillna(value=0) 

In [42]:
sets.count()

fecha_x                     2269046
person                      2472362
sku                         2269046
model                       2269046
condition                   2269046
storage                     2269046
color                       2269046
cant_eventos_x              2269046
compras_prod                2472362
vistas_prod                 2472362
otros                       2472362
fecha_y                     2471326
new_vs_returning            2471326
city                        2471326
region                      2471326
country                     2471326
device_type                 2471326
screen_resolution           2471309
operating_system_version    2471326
browser_version             2471326
cant_eventos_y              2471326
visitas_sitio               2472362
cant_compras                2472362
cant_vistas                 2472362
cant_otros                  2472362
cant_visitas_sitio          2472362
dtype: int64

In [43]:
#  Nos quedamos con cant_eventos_x (ya que son iguales con cant_eventos_y) convertimos a int y Rellenamos con 0
sets['cant_eventos'] = sets['cant_eventos_x']
sets['cant_eventos'] = sets['cant_eventos'].fillna(value=0) 
sets['cant_eventos'] = sets['cant_eventos'].astype('int')
sets.drop('cant_eventos_x',axis=1, inplace=True)
sets.drop('cant_eventos_y',axis=1, inplace=True)
sets.head()

,fecha_x,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros,...,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio,cant_eventos
0,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,1.0,1.0,1.0,...,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1.0,1.0,4.0,1.0,1.0,9
1,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,...,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,1.0,1.0,20.0,22.0,5.0,65
2,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,...,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,1.0,1.0,20.0,22.0,5.0,65
3,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,...,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,1.0,1.0,20.0,22.0,5.0,65
4,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,...,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,1.0,1.0,20.0,22.0,5.0,65


In [44]:
# Rellenamos model con 0 para luego hacer un split entre marca y modelo
sets['model'] = sets['model'].fillna(value='0') 

In [45]:
# Rellenamos los demas con ''
sets = sets.fillna(value='') 

In [46]:
sets.count()

fecha_x                     2472362
person                      2472362
sku                         2472362
model                       2472362
condition                   2472362
storage                     2472362
color                       2472362
compras_prod                2472362
vistas_prod                 2472362
otros                       2472362
fecha_y                     2472362
new_vs_returning            2472362
city                        2472362
region                      2472362
country                     2472362
device_type                 2472362
screen_resolution           2472362
operating_system_version    2472362
browser_version             2472362
visitas_sitio               2472362
cant_compras                2472362
cant_vistas                 2472362
cant_otros                  2472362
cant_visitas_sitio          2472362
cant_eventos                2472362
dtype: int64

In [47]:
sets.person.describe()

count      2472362
unique       38829
top       5059f7fd
freq         96465
Name: person, dtype: object

## Categorizacion de los features

In [48]:
# Rellenamos cantidad de eventos y convertimos a int
sets['cant_eventos'] = sets['cant_eventos'].fillna(value=0) 
sets['cant_eventos'] = sets['cant_eventos'].astype('int')

sets['cant_compras'] = sets['cant_compras'].fillna(value=0) 
sets['cant_compras'] = sets['cant_compras'].astype('int')
sets['cant_vistas'] = sets['cant_vistas'].fillna(value=0) 
sets['cant_vistas'] = sets['cant_vistas'].astype('int')
sets['cant_otros'] = sets['cant_otros'].fillna(value=0) 
sets['cant_otros'] = sets['cant_otros'].astype('int')
sets['cant_visitas_sitio'] = sets['cant_visitas_sitio'].fillna(value=0) 
sets['cant_visitas_sitio'] = sets['cant_visitas_sitio'].astype('int')

In [49]:
# Partimos el feature model en marca y modelo
sets['model'] = sets['model'].apply(lambda x: x.split(' '))
sets['marca'] = sets['model'].apply(lambda x: "apple" if str(x[0]) == "iPhone" else str(x[0]))
sets['modelo'] = sets['model'].apply(lambda x: ' '.join(x[1:]) if len(x)>1 else "0" )
sets.drop('model', axis=1, inplace=True)
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,operating_system_version,browser_version,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio,cant_eventos,marca,modelo
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1.0,1.0,1.0,2018-05-18 00:07:22,...,Android 7,Chrome Mobile 64.0,1.0,1,4,1,1,9,Samsung,Galaxy J7 Prime
1,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,2018-05-14 23:50:22,...,Android 5.1.1,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s
2,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,2018-05-16 02:48:13,...,Android 5.1.1,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s
3,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,2018-05-18 00:11:26,...,Android 5.1.1,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s
4,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,2018-05-18 22:11:46,...,Android 5.1.1,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s


In [50]:
# creo una nueva columna person_int para entrenar transformando el valor alfanumerico de person a numerico
sets['person_int'] = labelencoder.fit_transform(sets['person'])
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,browser_version,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio,cant_eventos,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1.0,1.0,1.0,2018-05-18 00:07:22,...,Chrome Mobile 64.0,1.0,1,4,1,1,9,Samsung,Galaxy J7 Prime,10956
1,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,2018-05-14 23:50:22,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
2,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,2018-05-16 02:48:13,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
3,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,2018-05-18 00:11:26,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
4,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1.0,1.0,1.0,2018-05-18 22:11:46,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263


In [51]:
# convertimos a integer el valor flotante de compras_prod
sets['compras_prod'] = sets['compras_prod'].astype('int')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,browser_version,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio,cant_eventos,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1,1.0,1.0,2018-05-18 00:07:22,...,Chrome Mobile 64.0,1.0,1,4,1,1,9,Samsung,Galaxy J7 Prime,10956
1,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1.0,1.0,2018-05-14 23:50:22,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
2,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1.0,1.0,2018-05-16 02:48:13,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
3,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1.0,1.0,2018-05-18 00:11:26,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
4,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1.0,1.0,2018-05-18 22:11:46,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263


In [52]:
# convertimos a integer el valor flotante de vistas_prod
sets['vistas_prod'] = sets['vistas_prod'].astype('int')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,browser_version,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio,cant_eventos,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1,1,1.0,2018-05-18 00:07:22,...,Chrome Mobile 64.0,1.0,1,4,1,1,9,Samsung,Galaxy J7 Prime,10956
1,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1.0,2018-05-14 23:50:22,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
2,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1.0,2018-05-16 02:48:13,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
3,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1.0,2018-05-18 00:11:26,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
4,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1.0,2018-05-18 22:11:46,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263


In [53]:
# convertimos a integer el valor flotante de otros
sets['otros'] = sets['otros'].astype('int')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,browser_version,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio,cant_eventos,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1,1,1,2018-05-18 00:07:22,...,Chrome Mobile 64.0,1.0,1,4,1,1,9,Samsung,Galaxy J7 Prime,10956
1,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-14 23:50:22,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
2,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-16 02:48:13,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
3,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-18 00:11:26,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263
4,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-18 22:11:46,...,Chrome Mobile 66.0,1.0,1,20,22,5,65,apple,5s,26263


In [54]:
# convertimos a integer el valor flotante de visitas_sitio
sets['visitas_sitio'] = sets['visitas_sitio'].astype('int')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,browser_version,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio,cant_eventos,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1,1,1,2018-05-18 00:07:22,...,Chrome Mobile 64.0,1,1,4,1,1,9,Samsung,Galaxy J7 Prime,10956
1,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-14 23:50:22,...,Chrome Mobile 66.0,1,1,20,22,5,65,apple,5s,26263
2,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-16 02:48:13,...,Chrome Mobile 66.0,1,1,20,22,5,65,apple,5s,26263
3,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-18 00:11:26,...,Chrome Mobile 66.0,1,1,20,22,5,65,apple,5s,26263
4,2018-05-14 23:54:19,ad93850f,304,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-18 22:11:46,...,Chrome Mobile 66.0,1,1,20,22,5,65,apple,5s,26263


In [55]:
# convertimos a string el valor flotante de sku
sets['sku'] = sets['sku'].astype('str')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,browser_version,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio,cant_eventos,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288.0,Excelente,32GB,Dourado,1,1,1,2018-05-18 00:07:22,...,Chrome Mobile 64.0,1,1,4,1,1,9,Samsung,Galaxy J7 Prime,10956
1,2018-05-14 23:54:19,ad93850f,304.0,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-14 23:50:22,...,Chrome Mobile 66.0,1,1,20,22,5,65,apple,5s,26263
2,2018-05-14 23:54:19,ad93850f,304.0,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-16 02:48:13,...,Chrome Mobile 66.0,1,1,20,22,5,65,apple,5s,26263
3,2018-05-14 23:54:19,ad93850f,304.0,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-18 00:11:26,...,Chrome Mobile 66.0,1,1,20,22,5,65,apple,5s,26263
4,2018-05-14 23:54:19,ad93850f,304.0,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-18 22:11:46,...,Chrome Mobile 66.0,1,1,20,22,5,65,apple,5s,26263


In [56]:
# Partimos el feature operating_system_version en nombre sistema op y browser_version  en nombre del browser
sets['operating_system_version'] = sets['operating_system_version'].apply(lambda x: x.split(' '))
sets['operating_system_version'] = sets['operating_system_version'].apply(lambda x: x[0])
sets['browser_version'] = sets['browser_version'].apply(lambda x: x.split(' '))
sets['browser_version'] = sets['browser_version'].apply(lambda x: x[0])
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,browser_version,visitas_sitio,cant_compras,cant_vistas,cant_otros,cant_visitas_sitio,cant_eventos,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288.0,Excelente,32GB,Dourado,1,1,1,2018-05-18 00:07:22,...,Chrome,1,1,4,1,1,9,Samsung,Galaxy J7 Prime,10956
1,2018-05-14 23:54:19,ad93850f,304.0,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-14 23:50:22,...,Chrome,1,1,20,22,5,65,apple,5s,26263
2,2018-05-14 23:54:19,ad93850f,304.0,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-16 02:48:13,...,Chrome,1,1,20,22,5,65,apple,5s,26263
3,2018-05-14 23:54:19,ad93850f,304.0,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-18 00:11:26,...,Chrome,1,1,20,22,5,65,apple,5s,26263
4,2018-05-14 23:54:19,ad93850f,304.0,Muito Bom,32GB,Cinza espacial,1,1,1,2018-05-18 22:11:46,...,Chrome,1,1,20,22,5,65,apple,5s,26263


In [57]:
#Categorizamos los valores de los features con label encoder
sets['sku'] = labelencoder.fit_transform(sets['sku'])
sets['marca'] = labelencoder.fit_transform(sets['marca'])
sets['modelo'] = labelencoder.fit_transform(sets['modelo'])
sets['condition'] = labelencoder.fit_transform(sets['condition'])
sets['storage'] = labelencoder.fit_transform(sets['storage'])
sets['color'] = labelencoder.fit_transform(sets['color'])
sets['new_vs_returning'] = labelencoder.fit_transform(sets['new_vs_returning'])
sets['city'] = labelencoder.fit_transform(sets['city'])
sets['region'] = labelencoder.fit_transform(sets['region'])
sets['country'] = labelencoder.fit_transform(sets['country'])
sets['device_type'] = labelencoder.fit_transform(sets['device_type'])
sets['screen_resolution'] = labelencoder.fit_transform(sets['screen_resolution'])
sets['operating_system_version'] = labelencoder.fit_transform(sets['operating_system_version'])
sets['browser_version'] = labelencoder.fit_transform(sets['browser_version'])
# sets['url'] = labelencoder.fit_transform(sets['url'])
# sets['skus'] = labelencoder.fit_transform(sets['skus'])
# sets['search_term'] = labelencoder.fit_transform(sets['search_term'])
# sets['staticpage'] = labelencoder.fit_transform(sets['staticpage'])
# sets['campaign_source'] = labelencoder.fit_transform(sets['campaign_source'])
# sets['search_engine'] = labelencoder.fit_transform(sets['search_engine'])
# sets['channel'] = labelencoder.fit_transform(sets['channel'])
# sets.head()

In [58]:
#Categorizamos la fecha del visto del producto en dia mes y dia de la semana  (ya que el anio es 2018)
sets['dia_view_prod'] = sets['fecha_x'].apply(lambda x: x.day if type(x) != str else 0).astype('int')
sets['mes_view_prod'] = sets['fecha_x'].apply(lambda x: x.month if type(x) != str else 0).astype('int')
sets['dia_sem_view_prod'] = sets['fecha_x'].apply(lambda x: x.dayofweek if type(x) != str else 0).astype('int')
sets = sets.drop('fecha_x',1)

In [59]:
#Categorizamos la fecha de visita del usuario en dia mes y dia de la semana (ya que el anio es 2018)
sets['dia_visit_user'] = sets['fecha_y'].apply(lambda x: x.day if type(x) != str else 0).astype('int')
sets['mes_visit_user'] = sets['fecha_y'].apply(lambda x: x.month if type(x) != str else 0).astype('int')
sets['dia_sem_visit_user'] = sets['fecha_y'].apply(lambda x: x.dayofweek if type(x) != str else 0).astype('int')
sets = sets.drop('fecha_y',1)

In [60]:
sets.cant_eventos.describe()

count    2.472362e+06
mean     4.572268e+02
std      6.390378e+02
min      0.000000e+00
25%      1.010000e+02
50%      2.670000e+02
75%      5.290000e+02
max      4.438000e+03
Name: cant_eventos, dtype: float64

In [61]:
prediction_set = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory = False)
training_set = pd.read_csv('data/labels_training_set.csv', low_memory=False)

print('PREDICCION ' , prediction_set.person.describe())
print('ENTRENAMIENTO ', training_set.person.describe())
print('ENTRENAMIENTO 1s ', training_set[training_set['label'] == 1].shape)
print('ENTRENAMIENTO 0s ', training_set[training_set['label'] == 0].shape)

prediction_set = prediction_set.merge(sets, on = 'person', how = 'inner')

training_set = training_set.merge(sets, on = 'person', how = 'inner')

PREDICCION  count        19415
unique       19415
top       44b95dc1
freq             1
Name: person, dtype: object
ENTRENAMIENTO  count        19414
unique       19414
top       703ea362
freq             1
Name: person, dtype: object
ENTRENAMIENTO 1s  (980, 2)
ENTRENAMIENTO 0s  (18434, 2)


In [63]:
prediction_set.person.describe()

count      1297430
unique       19415
top       5059f7fd
freq         96465
Name: person, dtype: object

In [64]:
training_set[training_set['label'] == 0].shape

(897495, 32)

In [65]:
prediction_set.shape

(1297430, 31)

In [66]:
training_set[training_set['label'] == 0].shape

(897495, 32)

In [67]:
# Guardar Set

In [68]:
training_set.to_csv('data/set_entrenamiento_25.csv', encoding='utf-8', index=False)

In [69]:
prediction_set.to_csv('data/set_prediccion_25.csv', encoding='utf-8', index=False)